### Prepare data

In [ ]:
import mne

data_path = r"data\first_try\mati\mati_test_real_movement_1_annotated.fif"
is_real_movement = "real" in data_path.lower()
recording_type = "real_movement" if is_real_movement else "motor_imagery"

raw = mne.io.read_raw_fif(data_path, preload=True)
eeg_channels = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15", "A16"]
raw.pick(picks=eeg_channels)
raw.resample(sfreq=250)
raw.filter(l_freq=1.0, h_freq=40.0, fir_design='firwin')
raw.notch_filter(freqs=[50.0])

mateusz_mapping = {
    'A1': 'Fp1',
    'A2': 'Fp2',
    'A3': 'F4',
    'A4': 'Fz',
    'A5': 'F3',
    'A6': 'T7',
    'A7': 'C3',
    'A8': 'Cz',
    'A9': 'C4',
    'A10': 'T8',
    'A11': 'P4',
    'A12': 'Pz',
    'A13': 'P3',
    'A14': 'O1',
    'A15': 'Oz',
    'A16': 'O2'
}

hania_mapping = {
    'A1': 'Fp1',
    'A2': 'Fp2',
    'A3': 'F4',
    'A4': 'Fz',
    'A5': 'F3',
    'A6': 'CP1', # modified
    'A7': 'C3', 
    'A8': 'Cz',
    'A9': 'C4',
    'A10': 'CP2', # modified
    'A11': 'P4',
    'A12': 'Pz',
    'A13': 'P3',
    'A14': 'FC1', # modified
    'A15': 'CPz', # modified // not sure if this is exactly correct
    'A16': 'FC2' # modified
}

raw.rename_channels(mateusz_mapping)
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)


### Show annotations

In [ ]:
all_events, all_events_id = mne.events_from_annotations(raw)
print(all_events)
print(all_events_id)

# rename event ids to more readable names
all_events_id = {'Both feets': 1, 'Both hands': 2, 'Left hand': 3, 'Relax': 4, 'Right hand': 5}


### Epoch data
There is issue with varying epoch-time - in our data relax takes 5s, and other samples 10s. MNE does not support multiple epoch times.\
Simple fix is to drop relax for now - we can create second epoch object with epochtime=5.

In [ ]:
if 'Relax' in all_events_id:
    relax_event_id = {'Relax': all_events_id['Relax']}
    del all_events_id['Relax']

task_margin = 1.0 # seconds, margin to be sure that we are focused on the task
task_end = 9.0 # seconds, to have same length for all epochs
epochs = mne.Epochs(
    raw=raw,
    events=all_events,
    event_id=all_events_id,
    baseline=None,
    tmin=task_margin,
    tmax=task_end,
    preload=True
)

task_end = 4.0
relax_epochs = mne.Epochs(
    raw=raw,
    events=all_events,
    event_id=relax_event_id,
    baseline=None,
    tmin=task_margin,
    tmax=task_end,
    preload=True
)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(5, 1, figsize=(10, 15))

for i, event_id in enumerate(all_events_id):
    epochs[event_id].average().plot(gfp=True, spatial_colors=True, titles={"eeg": event_id}, axes=axes[i], show=False)

relax_epochs.average().plot(gfp=True, spatial_colors=True, titles={"eeg": "Relax"}, axes=axes[4], show=False)

plt.tight_layout()
plt.show()


### Plot topography of each band for each state

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(15, 10))

for i, mode in enumerate(all_events_id):
    epochs[mode].compute_psd().plot_topomap(axes=axes[i], show=False)
    axes[i, 0].set_ylabel(mode, rotation=90, fontsize=12, fontweight='bold')

relax_epochs.compute_psd().plot_topomap(axes=axes[4], show=False)
axes[4, 0].set_ylabel('Relax', rotation=90, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


### Animate second epoch
For that, ffmpeg should be installed: https://www.ffmpeg.org/

In [ ]:
rec_time = task_end - task_margin
step = 0.004
slowdown = 20 # each frame will be displayed 20 times longer, 8 sec recording to 160 second animation
times = [v*step for v in range(int(task_margin/step), int(task_end/step))]
framerate = 1/(step*slowdown)
filenames = []
for mode in ["Left hand", "Right hand"]:
    recording = epochs[mode][1].average()
    fig, anim = recording.animate_topomap(times=times, ch_type="eeg", frame_rate=framerate, blit=False, show=False)
    filename = f"visualisation_out/{recording_type}_{mode}.mp4"
    anim.save(filename=filename, writer="ffmpeg")
    filenames.append(filename)
    print(f"end of saving [{mode}] animation")


### Combine two videos together

In [ ]:
import subprocess

def combine_videos(left_video_path, right_video_path, output_path):
    cmd = f'ffmpeg -i \"{left_video_path}\" -i \"{right_video_path}\" -filter_complex hstack \"{output_path}\"'
    subprocess.run(cmd, shell=True)

left_video = filenames[0]
right_video = filenames[1]
output_video = "visualisation_out/left_and_right_hand.mp4"

combine_videos(left_video, right_video, output_video)

### Show epoch's most active regions

In [ ]:
n_left = len(epochs['Left hand'])
n_right = len(epochs['Right hand'])
n_epochs = hania_mapping(n_left, n_right)

for i in range(n_epochs):
    fig, axes = plt.subplots(1, 3, figsize=(5, 2), width_ratios=[12, 12, 1])
    for ax, mode in zip(axes, ['Left hand', 'Right hand']):
        evoked = epochs[mode][i].average()
        evoked.plot_topomap(times=5, ch_type='eeg', axes=[ax, axes[2]], show=False, average=8)
        ax.set_title(f"{mode} - epoch {i}")
    plt.show()

### Show record's most active regions

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 2), width_ratios=[12, 12, 1])
recording = epochs[mode].average()
for ax, mode in zip(axes, ['Left hand', 'Right hand']):
    evoked = epochs[mode].average()
    evoked.plot_topomap(times=5, ch_type='eeg', axes=[ax, axes[2]], show=False, average=8)
    ax.set_title(f"{mode} - epochs average")

### Show step-by-step comparison

In [ ]:
step = 0.05
times = [v * step for v in range(int(task_margin / step), int(2.0 / step))]  # up to 2 second, as blink happened after that in this recording and it distorts the scale
n_cols = len(times) + 1  # extra column for colorbar
fig2, axes2 = plt.subplots(2, n_cols, figsize=(len(times) * 2, 4), width_ratios=[5] * len(times) + [1])
for i, mode_name in enumerate(['Left hand', 'Right hand']):
    evoked = epochs[mode_name].average()
    evoked.plot_topomap(times=times, ch_type="eeg", axes=axes2[i, :], show=False, average=step)
    axes2[i, 0].set_ylabel(mode_name, rotation=90, fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()